In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm  # Import tqdm for the progress bar

In [2]:

def generate_staggered_did_data_hete_effect(
    n_units=200,
    num_pre_periods=5,
    num_post_periods=5,
    linearity_degree=1,
    pre_trend_bias_delta=0.2,
    propensity_noise_scale=0.5, # Scale of noise added to utility for assignment randomness
    epsilon_scale=1,
    seed=42
):
    """
    Generates panel data for DiD with staggered adoption, propensity scores,
    fixed covariates (8 total, mixed static/dynamic), and HETEROGENEOUS
    treatment effects conditional on X1 (static Bern) and X3 (dynamic Cat).

    Covariates (8 total):
    - X1: Bernoulli(p=0.66) - STATIC, influences propensity & treatment effect
    - X2: Bernoulli(p=0.45) - Time-varying
    - X3: Categorical({1,2,3,4} p={0.3,0.1,0.2,0.4}) - Time-varying, influences treatment effect
    - X4-X7: Numerical (Normal(0,1)) - Time-varying
    - X8: Numerical (Normal(0,1)) - STATIC, influences propensity

    Treatment Effect Logic:
    - Base effect size ('base_beta') determined by linearity_degree.
    - Modifier term = sqrt(abs(X1))
    - If X3 is 1 or 3: Effect = 1.5 * modifier + base_beta
    - If X3 is 2:       Effect = base_beta
    - If X3 is 4:       Effect = base_beta - 0.5 * modifier

    Args:
        n_units (int): Total number of units.
        num_pre_periods (int): Periods before the *earliest* treatment.
        num_post_periods (int): Periods after the *earliest* treatment.
        linearity_degree (int): Degree of linearity in the DGP (1-4). Also sets base treatment effect.
        pre_trend_bias_delta (float): Bias for pre-trends in eventually treated groups.
        propensity_noise_scale (float): Std deviation of noise added to group utility.
        epsilon_scale (float): Std deviation of the outcome error term.
        seed (int): Random seed for reproducibility.

    Returns:
        pd.DataFrame: Generated panel data with heterogeneous treatment effects.
    """
    np.random.seed(seed)
    total_covariates = 8 # Fixed number of covariates

    # --- 1. Generate STATIC Unit-Level Covariates (X1, X8) ---
    unit_ids = np.arange(n_units)
    unit_X1_bern = np.random.binomial(n=1, p=0.66, size=n_units)
    unit_X8_num = np.random.normal(0, 1, size=n_units)

    # --- 2. Calculate Group Utilities and Assign Groups based on STATIC X1, X8 ---
    coeffs = {
        'g1': {'intercept': 0.1, 'x1_bern': 0.8, 'x8_num': 0.6},
        'g2': {'intercept': 0.0, 'x1_bern': -0.5, 'x8_num': -0.7},
        'g3': {'intercept': -0.1, 'x1_bern': 0.3, 'x8_num': 0.4}
    }
    V0 = np.zeros(n_units)
    V1 = coeffs['g1']['intercept'] + coeffs['g1']['x1_bern'] * unit_X1_bern + coeffs['g1']['x8_num'] * unit_X8_num
    V2 = coeffs['g2']['intercept'] + coeffs['g2']['x1_bern'] * unit_X1_bern + coeffs['g2']['x8_num'] * unit_X8_num
    V3 = coeffs['g3']['intercept'] + coeffs['g3']['x1_bern'] * unit_X1_bern + coeffs['g3']['x8_num'] * unit_X8_num
    noise = np.random.normal(0, propensity_noise_scale, size=(n_units, 4))
    U = np.column_stack((V0, V1, V2, V3)) + noise
    unit_treatment_group = np.argmax(U, axis=1)

    # --- 3. Create Panel DataFrame and Merge STATIC Unit-Level Info ---
    periods = num_pre_periods + num_post_periods
    time_periods = np.arange(periods)
    data = pd.DataFrame({
        'unit_id': np.repeat(unit_ids, periods),
        'time': np.tile(time_periods, n_units)
    })
    df_unit_static = pd.DataFrame({
        'unit_id': unit_ids,
        'treatment_group': unit_treatment_group,
        'X1': unit_X1_bern,
        'X8': unit_X8_num
    })
    data = pd.merge(data, df_unit_static, on='unit_id', how='left')

    # --- 4. Generate Time-Varying Covariates (X2-X7) ---
    n_observations = len(data)
    data['X2'] = np.random.binomial(n=1, p=0.45, size=n_observations)
    cat_choices = [1, 2, 3, 4]; cat_probs = [0.3, 0.1, 0.2, 0.4]
    data['X3'] = np.random.choice(cat_choices, size=n_observations, p=cat_probs)
    X_num_time_varying = np.random.normal(0, 1, size=(n_observations, 4))
    data['X4'] = X_num_time_varying[:, 0]
    data['X5'] = X_num_time_varying[:, 1]
    data['X6'] = X_num_time_varying[:, 2]
    data['X7'] = X_num_time_varying[:, 3]

    # --- 5. Define Treatment Timing and Indicators ---
    earliest_treatment_period = num_pre_periods
    conditions_timing = [
        data['treatment_group'] == 0, data['treatment_group'] == 1,
        data['treatment_group'] == 2, data['treatment_group'] == 3
    ]
    choices_timing = [ np.inf, earliest_treatment_period, earliest_treatment_period + 1, earliest_treatment_period + 2 ]
    data['first_treat_period'] = np.select(conditions_timing, choices_timing, default=np.nan)
    data['post_treatment'] = (data['time'] >= num_pre_periods).astype(int)
    data['eventually_treated'] = (data['treatment_group'] > 0).astype(int)
    data['D'] = (data['time'] >= data['first_treat_period']).astype(int)
    data['time_trend'] = data['time']

    # --- 6. Generate Outcome Variable (Y) with HETEROGENEOUS Effects ---

    # Determine BASE treatment effect based on linearity degree
    if linearity_degree == 1 or linearity_degree == 2: base_treatment_effect_beta = 3.0
    elif linearity_degree == 3: base_treatment_effect_beta = 5.0
    else: base_treatment_effect_beta = np.nan

    # Calculate the DYNAMIC treatment effect based on X1 and X3
    effect_modifier_term = np.sqrt(np.abs(data['X4'])) # sqrt(abs(Bernoulli)) is just the Bernoulli value itself, but use formula
    conditions_effect = [
        data['X3'].isin([1, 3]),
        data['X3'] == 2,
        data['X3'] == 4
    ]
    choices_effect = [
        1.5 * effect_modifier_term + base_treatment_effect_beta,
        base_treatment_effect_beta,
        base_treatment_effect_beta - 0.5 * effect_modifier_term
    ]
    data['dynamic_treatment_effect'] = np.select(conditions_effect, choices_effect, default=np.nan)

    # --- DGP Parameters (excluding treatment interaction) ---
    data['epsilon'] = np.random.normal(scale=epsilon_scale, size=len(data))

    beta_0 = -0.5 # Intercept
    beta_group_effect = 0.75 # Main effect of treated group (alpha_i)
    beta_time = 0.2 # Main effect of time trend (gamma_t)
    beta_x = np.array([-0.75, 0.5, -0.5, -1.30, 1.8, 2.5, -1.0, 0.3]) # Fixed covariate effects
    if len(beta_x) != total_covariates: raise ValueError("beta_x length mismatch")


    # Prepare covariate matrix X (order X1 to X8)
    X_cols = [f'X{i}' for i in range(1, total_covariates + 1)]
    X = data[X_cols].values

    # --- Calculate Y based on linearity_degree ---
    Y_base = (beta_0 + beta_group_effect * data['eventually_treated'] + beta_time * data['time_trend'])
    half = total_covariates // 2 # half = 4

    # Calculate Covariate Effects (Y_covariates) based on linearity degree
    if linearity_degree == 1: # Fully Linear
        Y_covariates = np.sum(beta_x * X, axis=1)
    elif linearity_degree == 2: # Half X non-linear
        Y_covariates = (np.sum(beta_x[:2] * (X[:, :2] ** 2), axis=1) +
                        np.sum(beta_x[2:4] * np.exp(X[:, 2:4]), axis=1) +
                        np.sum(beta_x[4:] * X[:, 4:], axis=1))
    elif linearity_degree == 3: 
        Y_base = (beta_0 + beta_group_effect * data['eventually_treated'] + beta_time * data['time_trend']**2)
        Y_covariates = (np.sum(beta_x[:2] * (X[:, :2] ** 2), axis=1) +
                        np.sum(beta_x[2:4] * np.exp(X[:, 2:4]), axis=1) +
                        np.sum(beta_x[4:6] * np.abs(X[:, 4:6]), axis=1) +
                        np.sum(beta_x[6:] * np.sqrt(np.abs(X[:, 6:])), axis=1))
    else:
         Y_covariates = 0


    Y_treatment = data['dynamic_treatment_effect'] * data['D']
        # Calculate CATE
    data['CATE'] = data['dynamic_treatment_effect'] * data['D']


    # Combine components for final Y
    data['Y'] = Y_base + Y_covariates + Y_treatment

    # --- Add pre-trend bias ---
    if pre_trend_bias_delta != 0:
        pre_period_mask = data['time'] < earliest_treatment_period
        bias_mask = pre_period_mask & (data['eventually_treated'] == 1)
        if linearity_degree == 3: # Trigger non-linear pre-trend if degree is 4
            seasonal_amplitude = 1.0; seasonal_period = 4
            seasonal_effect = seasonal_amplitude * np.sin(2 * np.pi * data['time'] / seasonal_period)
            data.loc[bias_mask, 'Y'] += pre_trend_bias_delta * seasonal_effect[bias_mask]
        else:
            time_diff = data['time'] - earliest_treatment_period
            data.loc[bias_mask, 'Y'] += pre_trend_bias_delta * time_diff[bias_mask]

    # Add final error term
    data['Y'] += data['epsilon']

    # --- 7. Finalize DataFrame ---
    final_cols = (['unit_id', 'time', 'treatment_group', 'first_treat_period', 'eventually_treated', 'D','post_treatment'] +
                   X_cols + ['dynamic_treatment_effect', 'Y', 'CATE', 'time_trend', 'epsilon']) # Add dynamic_treatment_effect column
    final_cols = [col for col in final_cols if col in data.columns]
    data = data[final_cols]

    return data



# Experiments

In [28]:
num_x_covariates = 6
linearity_degree=1

# Set the number of iterations and initialize the counter.
num_iterations = 100
count_at_least_two_non_significant = 0

num_pre_periods=4

num_post_periods=4
num_mcmc=500

epsilon_scale=1

number_of_groups=3

RMSE_per_group=np.zeros([num_iterations,number_of_groups])
MAE_per_group=np.zeros([num_iterations,number_of_groups])
MAPE_per_group=np.zeros([num_iterations,number_of_groups])

list_accumulated_p_values=[]

RMSE_overall=np.zeros([num_iterations])
MAE_overall=np.zeros([num_iterations])
MAPE_overall=np.zeros([num_iterations])

for i in tqdm(range(num_iterations), desc="Progress", unit="iteration"):
    # Generate a random seed for each iteration.
    seed_val = i

    # Generate data with specified hyperparameters.
    data_linear = generate_staggered_did_data_hete_effect(
        n_units=200,
        linearity_degree=linearity_degree,
        num_pre_periods=num_pre_periods,
        num_post_periods=num_post_periods,
        pre_trend_bias_delta=0,
        epsilon_scale=epsilon_scale,
        seed=seed_val
    )

    # Define the complex regression formula.
    formula_complex = "Y ~ eventually_treated + C(time) + C(time):eventually_treated + X1 + X2+X3+X4+ X5 + X6+X7+X8"
    
    # Fit the model.
    model_complex = smf.ols(formula=formula_complex, data=data_linear).fit()

    CATE_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)]['CATE'].index
    Pre_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==0)]['CATE'].index
    true_CATE=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)]['CATE']
    estimated_CATE=np.array(model_complex.params[[f'C(time)[T.{t}]:eventually_treated' for t in range(num_pre_periods, num_pre_periods + num_post_periods)]])
    final_CATE=np.zeros([int(len(Pre_indexes)/4),num_post_periods])
    
    for k in range(int(len(Pre_indexes)/4)):
        final_CATE[k,:]=estimated_CATE

    true_CATE=np.array(true_CATE).reshape(final_CATE.shape)
    accumulated_p_values=np.zeros([num_post_periods,int(len(Pre_indexes)/4)])


    for k in range(int(len(Pre_indexes)/4)):
        accumulated_p_values[:,k]=np.array(model_complex.pvalues[[f'C(time)[T.{t}]:eventually_treated' for t in range(num_pre_periods, num_pre_periods + num_post_periods)]])

    accumulated_p_values=accumulated_p_values.reshape(accumulated_p_values.shape[0]*accumulated_p_values.shape[1])
    list_accumulated_p_values.append(accumulated_p_values)

    RMSE_overall[i]=np.sqrt(np.mean((final_CATE-true_CATE)**2))
    MAE_overall[i]=np.mean(np.abs(final_CATE-true_CATE))

    # Find the indices where true_CATE is not zero.
    non_zero_indices = true_CATE != 0

    # Filter the arrays to include only non-zero true_CATE values.
    filtered_final_CATE = final_CATE[non_zero_indices]
    filtered_true_CATE = true_CATE[non_zero_indices]

    MAPE_overall[i] = np.mean(np.abs((filtered_final_CATE - filtered_true_CATE) / filtered_true_CATE))


    for group in range(number_of_groups):
        CATE_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)& (data_linear['treatment_group']==group+1)]['CATE'].index
        Pre_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==0)& (data_linear['treatment_group']==group+1)]['CATE'].index
        true_CATE=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)& (data_linear['treatment_group']==group+1)]['CATE']
        final_CATE=np.zeros([int(len(Pre_indexes)/4),num_post_periods])

        for k in range(int(len(Pre_indexes)/4)):
            final_CATE[k,:]=estimated_CATE

        true_CATE=np.array(true_CATE).reshape(final_CATE.shape)

        RMSE_per_group[i, group]=np.sqrt(np.mean((final_CATE-true_CATE)**2))
        MAE_per_group[i, group]=np.mean(np.abs(final_CATE-true_CATE))

        # Find the indices where true_CATE is not zero.
        non_zero_indices = true_CATE != 0

        # Filter the arrays to include only non-zero true_CATE values.
        filtered_final_CATE = final_CATE[non_zero_indices]
        filtered_true_CATE = true_CATE[non_zero_indices]

        MAPE_per_group[i, group] = np.mean(np.abs((filtered_final_CATE - filtered_true_CATE) / filtered_true_CATE))

mean_RMSE_overall=np.mean(RMSE_overall)
mean_MAE_overall=np.mean(MAE_overall)
mean_MAPE_overall=np.mean(MAPE_overall)
std_RMSE_overall=np.std(RMSE_overall)
std_MAE_overall=np.std(MAE_overall)
std_MAPE_overall=np.std(MAPE_overall)

print(f"Mean RMSE for {num_iterations} simulations: {mean_RMSE_overall}")
print(f"Standard Deviation RMSE for {num_iterations} simulations: {std_RMSE_overall}")
print(f"Mean MAE for {num_iterations} simulations: {mean_MAE_overall}")
print(f"Standard Deviation MAE for {num_iterations} simulations: {std_MAE_overall}")
print(f"Mean MAPE for {num_iterations} simulations: {mean_MAPE_overall}")
print(f"Standard Deviation MAPE for {num_iterations} simulations: {std_MAPE_overall}")

for h in range(number_of_groups):
    print(f"Mean RMSE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(RMSE_per_group[:,h])}")
    print(f"Standard Deviation RMSE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(RMSE_per_group[:,h])}")
    print(f"Mean MAE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(MAE_per_group[:,h])}")
    print(f"Standard Deviation MAE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(MAE_per_group[:,h])}")
    print(f"Mean MAPE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(MAPE_per_group[:,h])}")
    print(f"Standard Deviation MAPE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(MAPE_per_group[:,h])}")


# === Sheet 1: Metrics Data ===

# 1. Prepare data dictionary for metrics (overall and per_group)
metrics_data_dict = {
    'RMSE_overall': RMSE_overall,
    'MAE_overall': MAE_overall,
    'MAPE_overall': MAPE_overall,
}

# 2. Flatten the _per_group arrays into columns
for i in range(number_of_groups):
    metrics_data_dict[f'RMSE_group_{i}'] = RMSE_per_group[:, i]
    metrics_data_dict[f'MAE_group_{i}'] = MAE_per_group[:, i]
    metrics_data_dict[f'MAPE_group_{i}'] = MAPE_per_group[:, i]

# 3. Create the first DataFrame for metrics
df_metrics = pd.DataFrame(metrics_data_dict)

# === Sheet 2: P-Values Data (Handling Variable Lengths) ===

df_p_values = None # Initialize in case the list is empty

if not list_accumulated_p_values:
    print("Warning: 'list_accumulated_p_values' is empty. P-Value sheet will not be created.")
else:
    # 1. Find the maximum length of the p-value vectors
    max_len = 0
    for vec in list_accumulated_p_values:
        # Check if the element is actually a numpy array or list-like
        if hasattr(vec, '__len__'):
             max_len = max(max_len, len(vec))
        # else: handle potential non-iterable elements if necessary

    if max_len == 0 and list_accumulated_p_values:
         print("Warning: list_accumulated_p_values contains elements but none have length > 0.")
         # Decide how to handle this - maybe create an empty df?

    # 2. Create padded data
    padded_p_values = []
    for i, vec in enumerate(list_accumulated_p_values):
         # Ensure vec is treated as an iterable, default to empty if not applicable
        current_vec = []
        if hasattr(vec, '__len__'):
            current_vec = list(vec) # Convert numpy array to list for easy padding

        # Create a padded row with NaN for missing values
        padded_row = current_vec + [np.nan] * (max_len - len(current_vec))
        padded_p_values.append(padded_row)

    # 3. Create column names for the p-values sheet
    p_value_columns = [f'p_value_{i}' for i in range(max_len)]

    # 4. Create the second DataFrame for p-values
    df_p_values = pd.DataFrame(padded_p_values, columns=p_value_columns)

# === Save to Excel File with Multiple Sheets ===

output_filename_excel = 'OLS_CATE_GATE_PS_and_PValues_linearity=1.xlsx'

# Use ExcelWriter to write multiple DataFrames to different sheets
try:
    with pd.ExcelWriter(output_filename_excel, engine='openpyxl') as writer:
        # Write the metrics DataFrame to the first sheet
        df_metrics.to_excel(writer, sheet_name='Metrics', index=False, float_format='%.6f')
        print(f"Metrics data saved to sheet 'Metrics' in {output_filename_excel}")

        # Write the p-values DataFrame to the second sheet (if it exists)
        if df_p_values is not None:
            df_p_values.to_excel(writer, sheet_name='P_Values', index=False, float_format='%.6f')
            print(f"P-Values data saved to sheet 'P_Values' in {output_filename_excel}")
        else:
             # Optionally create an empty sheet or just skip
             print("P-Values sheet was not created as the source list was empty or contained no vectors.")


except ImportError:
    print("\nError: Cannot write Excel file. Please install the 'openpyxl' library.")
    print("You can install it using: pip install openpyxl")
except Exception as e:
    print(f"\nAn error occurred while writing the Excel file: {e}")


Progress: 100%|██████████| 100/100 [00:20<00:00,  4.85iteration/s]


Mean RMSE for 100 simulations: 1.3490813015017382
Standard Deviation RMSE for 100 simulations: 0.061483422128521696
Mean MAE for 100 simulations: 1.088450152773144
Standard Deviation MAE for 100 simulations: 0.049743192363011744
Mean MAPE for 100 simulations: 0.25696227409212324
Standard Deviation MAPE for 100 simulations: 0.020939085387034494
Mean RMSE for 100 simulations for post-treatment period 1: 1.163629368560935
Standard Deviation RMSE for 100 simulations for post-treatment period 1: 0.14040160178216424
Mean MAE for 100 simulations for post-treatment period 1: 0.9452784403045068
Standard Deviation MAE for 100 simulations for post-treatment period 1: 0.10090994447329024
Mean MAPE for 100 simulations for post-treatment period 1: 0.2642228554940214
Standard Deviation MAPE for 100 simulations for post-treatment period 1: 0.02501477660145916
Mean RMSE for 100 simulations for post-treatment period 2: 1.3906331624647876
Standard Deviation RMSE for 100 simulations for post-treatment per

In [29]:
num_x_covariates = 6
linearity_degree=2

# Set the number of iterations and initialize the counter.
num_iterations = 100
count_at_least_two_non_significant = 0

num_pre_periods=4

num_post_periods=4
num_mcmc=500

epsilon_scale=1

number_of_groups=3

RMSE_per_group=np.zeros([num_iterations,number_of_groups])
MAE_per_group=np.zeros([num_iterations,number_of_groups])
MAPE_per_group=np.zeros([num_iterations,number_of_groups])

list_accumulated_p_values=[]

RMSE_overall=np.zeros([num_iterations])
MAE_overall=np.zeros([num_iterations])
MAPE_overall=np.zeros([num_iterations])

for i in tqdm(range(num_iterations), desc="Progress", unit="iteration"):
    # Generate a random seed for each iteration.
    seed_val = i

    # Generate data with specified hyperparameters.
    data_linear = generate_staggered_did_data_hete_effect(
        n_units=200,
        linearity_degree=linearity_degree,
        num_pre_periods=num_pre_periods,
        num_post_periods=num_post_periods,
        pre_trend_bias_delta=0,
        epsilon_scale=epsilon_scale,
        seed=seed_val
    )

    # Define the complex regression formula.
    formula_complex = "Y ~ eventually_treated + C(time) + C(time):eventually_treated + X1 + X2+X3+X4+ X5 + X6+X7+X8"
    
    # Fit the model.
    model_complex = smf.ols(formula=formula_complex, data=data_linear).fit()

    CATE_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)]['CATE'].index
    Pre_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==0)]['CATE'].index
    true_CATE=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)]['CATE']
    estimated_CATE=np.array(model_complex.params[[f'C(time)[T.{t}]:eventually_treated' for t in range(num_pre_periods, num_pre_periods + num_post_periods)]])
    final_CATE=np.zeros([int(len(Pre_indexes)/4),num_post_periods])
    
    for k in range(int(len(Pre_indexes)/4)):
        final_CATE[k,:]=estimated_CATE

    true_CATE=np.array(true_CATE).reshape(final_CATE.shape)
    accumulated_p_values=np.zeros([num_post_periods,int(len(Pre_indexes)/4)])


    for k in range(int(len(Pre_indexes)/4)):
        accumulated_p_values[:,k]=np.array(model_complex.pvalues[[f'C(time)[T.{t}]:eventually_treated' for t in range(num_pre_periods, num_pre_periods + num_post_periods)]])

    accumulated_p_values=accumulated_p_values.reshape(accumulated_p_values.shape[0]*accumulated_p_values.shape[1])
    list_accumulated_p_values.append(accumulated_p_values)

    RMSE_overall[i]=np.sqrt(np.mean((final_CATE-true_CATE)**2))
    MAE_overall[i]=np.mean(np.abs(final_CATE-true_CATE))

    # Find the indices where true_CATE is not zero.
    non_zero_indices = true_CATE != 0

    # Filter the arrays to include only non-zero true_CATE values.
    filtered_final_CATE = final_CATE[non_zero_indices]
    filtered_true_CATE = true_CATE[non_zero_indices]

    MAPE_overall[i] = np.mean(np.abs((filtered_final_CATE - filtered_true_CATE) / filtered_true_CATE))


    for group in range(number_of_groups):
        CATE_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)& (data_linear['treatment_group']==group+1)]['CATE'].index
        Pre_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==0)& (data_linear['treatment_group']==group+1)]['CATE'].index
        true_CATE=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)& (data_linear['treatment_group']==group+1)]['CATE']
        final_CATE=np.zeros([int(len(Pre_indexes)/4),num_post_periods])

        for k in range(int(len(Pre_indexes)/4)):
            final_CATE[k,:]=estimated_CATE

        true_CATE=np.array(true_CATE).reshape(final_CATE.shape)

        RMSE_per_group[i, group]=np.sqrt(np.mean((final_CATE-true_CATE)**2))
        MAE_per_group[i, group]=np.mean(np.abs(final_CATE-true_CATE))

        # Find the indices where true_CATE is not zero.
        non_zero_indices = true_CATE != 0

        # Filter the arrays to include only non-zero true_CATE values.
        filtered_final_CATE = final_CATE[non_zero_indices]
        filtered_true_CATE = true_CATE[non_zero_indices]

        MAPE_per_group[i, group] = np.mean(np.abs((filtered_final_CATE - filtered_true_CATE) / filtered_true_CATE))

mean_RMSE_overall=np.mean(RMSE_overall)
mean_MAE_overall=np.mean(MAE_overall)
mean_MAPE_overall=np.mean(MAPE_overall)
std_RMSE_overall=np.std(RMSE_overall)
std_MAE_overall=np.std(MAE_overall)
std_MAPE_overall=np.std(MAPE_overall)

print(f"Mean RMSE for {num_iterations} simulations: {mean_RMSE_overall}")
print(f"Standard Deviation RMSE for {num_iterations} simulations: {std_RMSE_overall}")
print(f"Mean MAE for {num_iterations} simulations: {mean_MAE_overall}")
print(f"Standard Deviation MAE for {num_iterations} simulations: {std_MAE_overall}")
print(f"Mean MAPE for {num_iterations} simulations: {mean_MAPE_overall}")
print(f"Standard Deviation MAPE for {num_iterations} simulations: {std_MAPE_overall}")

for h in range(number_of_groups):
    print(f"Mean RMSE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(RMSE_per_group[:,h])}")
    print(f"Standard Deviation RMSE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(RMSE_per_group[:,h])}")
    print(f"Mean MAE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(MAE_per_group[:,h])}")
    print(f"Standard Deviation MAE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(MAE_per_group[:,h])}")
    print(f"Mean MAPE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(MAPE_per_group[:,h])}")
    print(f"Standard Deviation MAPE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(MAPE_per_group[:,h])}")


# === Sheet 1: Metrics Data ===

# 1. Prepare data dictionary for metrics (overall and per_group)
metrics_data_dict = {
    'RMSE_overall': RMSE_overall,
    'MAE_overall': MAE_overall,
    'MAPE_overall': MAPE_overall,
}

# 2. Flatten the _per_group arrays into columns
for i in range(number_of_groups):
    metrics_data_dict[f'RMSE_group_{i}'] = RMSE_per_group[:, i]
    metrics_data_dict[f'MAE_group_{i}'] = MAE_per_group[:, i]
    metrics_data_dict[f'MAPE_group_{i}'] = MAPE_per_group[:, i]

# 3. Create the first DataFrame for metrics
df_metrics = pd.DataFrame(metrics_data_dict)

# === Sheet 2: P-Values Data (Handling Variable Lengths) ===

df_p_values = None # Initialize in case the list is empty

if not list_accumulated_p_values:
    print("Warning: 'list_accumulated_p_values' is empty. P-Value sheet will not be created.")
else:
    # 1. Find the maximum length of the p-value vectors
    max_len = 0
    for vec in list_accumulated_p_values:
        # Check if the element is actually a numpy array or list-like
        if hasattr(vec, '__len__'):
             max_len = max(max_len, len(vec))
        # else: handle potential non-iterable elements if necessary

    if max_len == 0 and list_accumulated_p_values:
         print("Warning: list_accumulated_p_values contains elements but none have length > 0.")
         # Decide how to handle this - maybe create an empty df?

    # 2. Create padded data
    padded_p_values = []
    for i, vec in enumerate(list_accumulated_p_values):
         # Ensure vec is treated as an iterable, default to empty if not applicable
        current_vec = []
        if hasattr(vec, '__len__'):
            current_vec = list(vec) # Convert numpy array to list for easy padding

        # Create a padded row with NaN for missing values
        padded_row = current_vec + [np.nan] * (max_len - len(current_vec))
        padded_p_values.append(padded_row)

    # 3. Create column names for the p-values sheet
    p_value_columns = [f'p_value_{i}' for i in range(max_len)]

    # 4. Create the second DataFrame for p-values
    df_p_values = pd.DataFrame(padded_p_values, columns=p_value_columns)

# === Save to Excel File with Multiple Sheets ===

output_filename_excel = 'OLS_CATE_GATE_PS_and_PValues_linearity=2.xlsx'

# Use ExcelWriter to write multiple DataFrames to different sheets
try:
    with pd.ExcelWriter(output_filename_excel, engine='openpyxl') as writer:
        # Write the metrics DataFrame to the first sheet
        df_metrics.to_excel(writer, sheet_name='Metrics', index=False, float_format='%.6f')
        print(f"Metrics data saved to sheet 'Metrics' in {output_filename_excel}")

        # Write the p-values DataFrame to the second sheet (if it exists)
        if df_p_values is not None:
            df_p_values.to_excel(writer, sheet_name='P_Values', index=False, float_format='%.6f')
            print(f"P-Values data saved to sheet 'P_Values' in {output_filename_excel}")
        else:
             # Optionally create an empty sheet or just skip
             print("P-Values sheet was not created as the source list was empty or contained no vectors.")


except ImportError:
    print("\nError: Cannot write Excel file. Please install the 'openpyxl' library.")
    print("You can install it using: pip install openpyxl")
except Exception as e:
    print(f"\nAn error occurred while writing the Excel file: {e}")


Progress: 100%|██████████| 100/100 [00:20<00:00,  4.95iteration/s]


Mean RMSE for 100 simulations: 1.821247102782803
Standard Deviation RMSE for 100 simulations: 0.3651001125336943
Mean MAE for 100 simulations: 1.4898608226539756
Standard Deviation MAE for 100 simulations: 0.34659916381797895
Mean MAPE for 100 simulations: 0.4051615331947974
Standard Deviation MAPE for 100 simulations: 0.1317573252345663
Mean RMSE for 100 simulations for post-treatment period 1: 1.6457366527514825
Standard Deviation RMSE for 100 simulations for post-treatment period 1: 0.4728459808649677
Mean MAE for 100 simulations for post-treatment period 1: 1.3717496212706275
Standard Deviation MAE for 100 simulations for post-treatment period 1: 0.4334101772799594
Mean MAPE for 100 simulations for post-treatment period 1: 0.40955538753152854
Standard Deviation MAPE for 100 simulations for post-treatment period 1: 0.1327514010510527
Mean RMSE for 100 simulations for post-treatment period 2: 1.8449328441918278
Standard Deviation RMSE for 100 simulations for post-treatment period 2: 

In [31]:
num_x_covariates = 6
linearity_degree=3

# Set the number of iterations and initialize the counter.
num_iterations = 100
count_at_least_two_non_significant = 0

num_pre_periods=4

num_post_periods=4
num_mcmc=500

epsilon_scale=1

number_of_groups=3

RMSE_per_group=np.zeros([num_iterations,number_of_groups])
MAE_per_group=np.zeros([num_iterations,number_of_groups])
MAPE_per_group=np.zeros([num_iterations,number_of_groups])

list_accumulated_p_values=[]

RMSE_overall=np.zeros([num_iterations])
MAE_overall=np.zeros([num_iterations])
MAPE_overall=np.zeros([num_iterations])

for i in tqdm(range(num_iterations), desc="Progress", unit="iteration"):
    # Generate a random seed for each iteration.
    seed_val = i

    # Generate data with specified hyperparameters.
    data_linear = generate_staggered_did_data_hete_effect(
        n_units=200,
        linearity_degree=linearity_degree,
        num_pre_periods=num_pre_periods,
        num_post_periods=num_post_periods,
        pre_trend_bias_delta=0,
        epsilon_scale=epsilon_scale,
        seed=seed_val
    )

    # Define the complex regression formula.
    formula_complex = "Y ~ eventually_treated + C(time) + C(time):eventually_treated + X1 + X2+X3+X4+ X5 + X6+X7+X8"
    
    # Fit the model.
    model_complex = smf.ols(formula=formula_complex, data=data_linear).fit()

    CATE_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)]['CATE'].index
    Pre_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==0)]['CATE'].index
    true_CATE=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)]['CATE']
    estimated_CATE=np.array(model_complex.params[[f'C(time)[T.{t}]:eventually_treated' for t in range(num_pre_periods, num_pre_periods + num_post_periods)]])
    final_CATE=np.zeros([int(len(Pre_indexes)/4),num_post_periods])
    
    for k in range(int(len(Pre_indexes)/4)):
        final_CATE[k,:]=estimated_CATE

    true_CATE=np.array(true_CATE).reshape(final_CATE.shape)
    accumulated_p_values=np.zeros([num_post_periods,int(len(Pre_indexes)/4)])


    for k in range(int(len(Pre_indexes)/4)):
        accumulated_p_values[:,k]=np.array(model_complex.pvalues[[f'C(time)[T.{t}]:eventually_treated' for t in range(num_pre_periods, num_pre_periods + num_post_periods)]])

    accumulated_p_values=accumulated_p_values.reshape(accumulated_p_values.shape[0]*accumulated_p_values.shape[1])
    list_accumulated_p_values.append(accumulated_p_values)

    RMSE_overall[i]=np.sqrt(np.mean((final_CATE-true_CATE)**2))
    MAE_overall[i]=np.mean(np.abs(final_CATE-true_CATE))

    # Find the indices where true_CATE is not zero.
    non_zero_indices = true_CATE != 0

    # Filter the arrays to include only non-zero true_CATE values.
    filtered_final_CATE = final_CATE[non_zero_indices]
    filtered_true_CATE = true_CATE[non_zero_indices]

    MAPE_overall[i] = np.mean(np.abs((filtered_final_CATE - filtered_true_CATE) / filtered_true_CATE))


    for group in range(number_of_groups):
        CATE_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)& (data_linear['treatment_group']==group+1)]['CATE'].index
        Pre_indexes=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==0)& (data_linear['treatment_group']==group+1)]['CATE'].index
        true_CATE=data_linear[(data_linear['eventually_treated']==1) & (data_linear['post_treatment']==1)& (data_linear['treatment_group']==group+1)]['CATE']
        final_CATE=np.zeros([int(len(Pre_indexes)/4),num_post_periods])

        for k in range(int(len(Pre_indexes)/4)):
            final_CATE[k,:]=estimated_CATE

        true_CATE=np.array(true_CATE).reshape(final_CATE.shape)

        RMSE_per_group[i, group]=np.sqrt(np.mean((final_CATE-true_CATE)**2))
        MAE_per_group[i, group]=np.mean(np.abs(final_CATE-true_CATE))

        # Find the indices where true_CATE is not zero.
        non_zero_indices = true_CATE != 0

        # Filter the arrays to include only non-zero true_CATE values.
        filtered_final_CATE = final_CATE[non_zero_indices]
        filtered_true_CATE = true_CATE[non_zero_indices]

        MAPE_per_group[i, group] = np.mean(np.abs((filtered_final_CATE - filtered_true_CATE) / filtered_true_CATE))

mean_RMSE_overall=np.mean(RMSE_overall)
mean_MAE_overall=np.mean(MAE_overall)
mean_MAPE_overall=np.mean(MAPE_overall)
std_RMSE_overall=np.std(RMSE_overall)
std_MAE_overall=np.std(MAE_overall)
std_MAPE_overall=np.std(MAPE_overall)

print(f"Mean RMSE for {num_iterations} simulations: {mean_RMSE_overall}")
print(f"Standard Deviation RMSE for {num_iterations} simulations: {std_RMSE_overall}")
print(f"Mean MAE for {num_iterations} simulations: {mean_MAE_overall}")
print(f"Standard Deviation MAE for {num_iterations} simulations: {std_MAE_overall}")
print(f"Mean MAPE for {num_iterations} simulations: {mean_MAPE_overall}")
print(f"Standard Deviation MAPE for {num_iterations} simulations: {std_MAPE_overall}")

for h in range(number_of_groups):
    print(f"Mean RMSE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(RMSE_per_group[:,h])}")
    print(f"Standard Deviation RMSE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(RMSE_per_group[:,h])}")
    print(f"Mean MAE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(MAE_per_group[:,h])}")
    print(f"Standard Deviation MAE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(MAE_per_group[:,h])}")
    print(f"Mean MAPE for {num_iterations} simulations for post-treatment period {h+1}: {np.mean(MAPE_per_group[:,h])}")
    print(f"Standard Deviation MAPE for {num_iterations} simulations for post-treatment period {h+1}: {np.std(MAPE_per_group[:,h])}")


# === Sheet 1: Metrics Data ===

# 1. Prepare data dictionary for metrics (overall and per_group)
metrics_data_dict = {
    'RMSE_overall': RMSE_overall,
    'MAE_overall': MAE_overall,
    'MAPE_overall': MAPE_overall,
}

# 2. Flatten the _per_group arrays into columns
for i in range(number_of_groups):
    metrics_data_dict[f'RMSE_group_{i}'] = RMSE_per_group[:, i]
    metrics_data_dict[f'MAE_group_{i}'] = MAE_per_group[:, i]
    metrics_data_dict[f'MAPE_group_{i}'] = MAPE_per_group[:, i]

# 3. Create the first DataFrame for metrics
df_metrics = pd.DataFrame(metrics_data_dict)

# === Sheet 2: P-Values Data (Handling Variable Lengths) ===

df_p_values = None # Initialize in case the list is empty

if not list_accumulated_p_values:
    print("Warning: 'list_accumulated_p_values' is empty. P-Value sheet will not be created.")
else:
    # 1. Find the maximum length of the p-value vectors
    max_len = 0
    for vec in list_accumulated_p_values:
        # Check if the element is actually a numpy array or list-like
        if hasattr(vec, '__len__'):
             max_len = max(max_len, len(vec))
        # else: handle potential non-iterable elements if necessary

    if max_len == 0 and list_accumulated_p_values:
         print("Warning: list_accumulated_p_values contains elements but none have length > 0.")
         # Decide how to handle this - maybe create an empty df?

    # 2. Create padded data
    padded_p_values = []
    for i, vec in enumerate(list_accumulated_p_values):
         # Ensure vec is treated as an iterable, default to empty if not applicable
        current_vec = []
        if hasattr(vec, '__len__'):
            current_vec = list(vec) # Convert numpy array to list for easy padding

        # Create a padded row with NaN for missing values
        padded_row = current_vec + [np.nan] * (max_len - len(current_vec))
        padded_p_values.append(padded_row)

    # 3. Create column names for the p-values sheet
    p_value_columns = [f'p_value_{i}' for i in range(max_len)]

    # 4. Create the second DataFrame for p-values
    df_p_values = pd.DataFrame(padded_p_values, columns=p_value_columns)

# === Save to Excel File with Multiple Sheets ===

output_filename_excel = 'OLS_CATE_GATE_PS_and_PValues_linearity=3.xlsx'

# Use ExcelWriter to write multiple DataFrames to different sheets
try:
    with pd.ExcelWriter(output_filename_excel, engine='openpyxl') as writer:
        # Write the metrics DataFrame to the first sheet
        df_metrics.to_excel(writer, sheet_name='Metrics', index=False, float_format='%.6f')
        print(f"Metrics data saved to sheet 'Metrics' in {output_filename_excel}")

        # Write the p-values DataFrame to the second sheet (if it exists)
        if df_p_values is not None:
            df_p_values.to_excel(writer, sheet_name='P_Values', index=False, float_format='%.6f')
            print(f"P-Values data saved to sheet 'P_Values' in {output_filename_excel}")
        else:
             # Optionally create an empty sheet or just skip
             print("P-Values sheet was not created as the source list was empty or contained no vectors.")


except ImportError:
    print("\nError: Cannot write Excel file. Please install the 'openpyxl' library.")
    print("You can install it using: pip install openpyxl")
except Exception as e:
    print(f"\nAn error occurred while writing the Excel file: {e}")


Progress: 100%|██████████| 100/100 [00:17<00:00,  5.68iteration/s]


Mean RMSE for 100 simulations: 2.2959979913729054
Standard Deviation RMSE for 100 simulations: 0.39908203081951427
Mean MAE for 100 simulations: 1.8355680176056979
Standard Deviation MAE for 100 simulations: 0.40880799005187146
Mean MAPE for 100 simulations: 0.2885905871369962
Standard Deviation MAPE for 100 simulations: 0.10184667159775726
Mean RMSE for 100 simulations for post-treatment period 1: 1.942044514599113
Standard Deviation RMSE for 100 simulations for post-treatment period 1: 0.6424829892115806
Mean MAE for 100 simulations for post-treatment period 1: 1.629457764780216
Standard Deviation MAE for 100 simulations for post-treatment period 1: 0.5961173908787963
Mean MAPE for 100 simulations for post-treatment period 1: 0.2986071241002019
Standard Deviation MAPE for 100 simulations for post-treatment period 1: 0.10820214083701167
Mean RMSE for 100 simulations for post-treatment period 2: 2.3578950352825974
Standard Deviation RMSE for 100 simulations for post-treatment period 2: